<a href="https://colab.research.google.com/github/auqa5325/AUQA-colab/blob/main/AUQA_questify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3 -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.0 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

AWS_ACCESS_KEY_ID     = userdata.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
#AWS_SESSION_TOKEN     = userdata.get("AWS_SESSION_TOKEN")  # may be None
AWS_REGION = userdata.get("AWS_REGION")

os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
os.environ["AWS_REGION"] = AWS_REGION

print("✅ Credentials set. Region:", AWS_REGION)


✅ Credentials set. Region: ap-south-1


In [3]:
import boto3, os

REGION = os.environ["AWS_REGION"]
session = boto3.Session(region_name=REGION)
bedrock         = session.client("bedrock")
bedrock_runtime = session.client("bedrock-runtime")
print("✅ boto3 session initialized in:", session.region_name)


✅ boto3 session initialized in: ap-south-1


In [4]:
import boto3
s = boto3.Session(region_name="ap-south-1")
print(s.client("sts").get_caller_identity())   # must print Account, Arn
creds = s.get_credentials().get_frozen_credentials()
print("AccessKey:", creds.access_key[:4], "HasToken:", bool(creds.token))


{'UserId': 'AIDA4L4FNUDYI73ZUG6BD', 'Account': '850146468080', 'Arn': 'arn:aws:iam::850146468080:user/Anna_university_L1', 'ResponseMetadata': {'RequestId': '47d69c2b-c3ec-4e7f-836f-2c1f10053209', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '47d69c2b-c3ec-4e7f-836f-2c1f10053209', 'x-amz-sts-extended-request-id': 'MTphcC1zb3V0aC0xOjE3NTY1NDg4MzQxNDA6Ujp3Y0FQd1kzUA==', 'content-type': 'text/xml', 'content-length': '415', 'date': 'Sat, 30 Aug 2025 10:13:54 GMT'}, 'RetryAttempts': 0}}
AccessKey: AKIA HasToken: False


# output dict structure :
* id → unique identifier for this response (good for
logging/debugging).

* type → "message" → tells you this is a message object.

* role → "assistant" → the speaker role (assistant vs. user).

* model → which model gave this reply (claude-3-sonnet-20240229).

* content → a list of parts that make up the response.

* stop_reason → why the model stopped (e.g., end_turn, max_tokens).

* stop_sequence → custom sequence that stopped generation (here it’s None).

* usage → token usage info (handy for cost + rate limits).

In [ ]:
import json
model_id = "mistral.mixtral-8x7b-instruct-v0:1"
prompt = "explain tcp ip?"

body = {
    "prompt": prompt,
    "max_tokens": 100,
}

resp = bedrock_runtime.invoke_model(modelId=model_id, body=json.dumps(body))
output = json.loads(resp["body"].read())
print(output["outputs"][0]["text"])


In [4]:
import boto3
session = boto3.Session(region_name="ap-south-1")
sts = session.client("sts")
print(sts.get_caller_identity())  # should return Account, Arn, UserId


{'UserId': 'AIDA4L4FNUDYI73ZUG6BD', 'Account': '850146468080', 'Arn': 'arn:aws:iam::850146468080:user/Anna_university_L1', 'ResponseMetadata': {'RequestId': '0318db39-9d37-4c6e-bb25-6767026e89f2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0318db39-9d37-4c6e-bb25-6767026e89f2', 'x-amz-sts-extended-request-id': 'MTphcC1zb3V0aC0xOjE3NTY0NTkzNTE3NDk6UjpJd1BrYlNOVg==', 'content-type': 'text/xml', 'content-length': '415', 'date': 'Fri, 29 Aug 2025 09:22:31 GMT'}, 'RetryAttempts': 0}}


{'outputs': [{'text': '\n\nTCP/IP is a suite of protocols that defines the Internet. Originally designed for the UNIX operating system. The TCP/IP protocol suite is a four-layer model that provides end-to-end connectivity specifying how data should be packetized, addressed, transmitted, routed, and received.\n\nThe four layers of the TCP/IP model are:\n\n1. The Application Layer: This is the topmost layer of the TCP/IP', 'stop_reason': 'length'}]}

In [5]:
!pip -q install opensearch-py requests-aws4auth

from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

# --- CONFIG ---
host   = "https://search-test1-annauniv-pcx3f52wxykhpd4md6v4bjeqdy.ap-south-1.es.amazonaws.com"  # OpenSearch domain endpoint
region = "ap-south-1"
service = "es"

# --- AWS SigV4 Auth ---
session = boto3.Session()
credentials = session.get_credentials()
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    service,
    session_token=credentials.token
)

# --- OpenSearch Client ---
client = OpenSearch(
    hosts=[host],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

# --- TEST: Get cluster info ---
print(client.info())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.5/371.5 kB 7.4 MB/s eta 0:00:00
{'name': 'd69eb8cacc6e2cf42750a5f4788db327', 'cluster_name': '850146468080:test1-annauniv', 'cluster_uuid': 'qhkHo9X9Suuw8REB9uInGQ', 'version': {'distribution': 'opensearch', 'number': '2.19.0', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2025-07-24T06:15:41.026838036Z', 'build_snapshot': False, 'lucene_version': '9.12.1', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


In [106]:
"""
index_name = "test-auqa"

index_body = {
    "settings": {
        "index": {
            "knn": True,
            "knn.algo_param.ef_search": 512,
            "number_of_shards": 5,
            "number_of_replicas": 1
        }
    },
    "mappings": {
        "properties": {
            # main chunk text
            "chunk_text": {
                "type": "text",
                "fields": {
                    "keyword": {"type": "keyword", "ignore_above": 256}
                }
            },

            # ids & names
            "course_id": {
                "type": "keyword"  # IDs are better stored as keyword for exact matches
            },

            "filename": {
                "type": "keyword"  # filenames usually exact matches
            },

            # single-page (legacy support)
            "page_no": {"type": "long"},

            # NEW: page range string (e.g., "1-6")
            "page_range": {"type": "keyword"},

            # NEW: array of page numbers (e.g., [1,2,3,4,5,6])
            "pages": {"type": "long"},

            # vector index used for ANN similarity search
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "nmslib",
                    "parameters": { "ef_construction": 512, "m": 16 }
                }
            }
        }
    }
}

# recreate index
if client.indices.exists(index=index_name):
    print(f"Index '{index_name}' already exists. Deleting and recreating...")
    client.indices.delete(index=index_name)

client.indices.create(index=index_name, body=index_body)
print(f"✅ Index '{index_name}' created successfully!")
"""

'\nindex_name = "test-auqa"\n\nindex_body = {\n    "settings": {\n        "index": {\n            "knn": True,\n            "knn.algo_param.ef_search": 512,\n            "number_of_shards": 5,\n            "number_of_replicas": 1\n        }\n    },\n    "mappings": {\n        "properties": {\n            # main chunk text\n            "chunk_text": {\n                "type": "text",\n                "fields": {\n                    "keyword": {"type": "keyword", "ignore_above": 256}\n                }\n            },\n\n            # ids & names\n            "course_id": {\n                "type": "keyword"  # IDs are better stored as keyword for exact matches\n            },\n           \n            "filename": {\n                "type": "keyword"  # filenames usually exact matches\n            },\n\n            # single-page (legacy support)\n            "page_no": {"type": "long"},\n\n            # NEW: page range string (e.g., "1-6")\n            "page_range": {"type": "keyword"},

{'test-annauniv': {'aliases': {}, 'mappings': {'properties': {'course_id': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'embedding': {'type': 'float'}, 'metadata': {'properties': {'page': {'type': 'long'}, 'source': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}, 'page_no': {'type': 'long'}, 'text': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'vector_field': {'type': 'knn_vector', 'dimension': 1024, 'method': {'engine': 'nmslib', 'space_type': 'cosinesimil', 'name': 'hnsw', 'parameters': {'ef_construction': 512, 'm': 16}}}}}, 'settings': {'index': {'replication': {'type': 'DOCUMENT'}, 'refresh_interval': '1s', 'number_of_shards': '5', 'knn.algo_param': {'ef_search': '512'}, 'provided_name': 'test-annauniv', 'knn': 'true', 'creation_date': '1756144596121', 'number_of_replicas': '1', 'uuid': 'rumasDZaQC6xBy4G48v1KQ', 'version': {'created': '136407827'}}}}}



In [10]:
import boto3, time, re

BUCKET = "anna-univ-qna"
DOCUMENT = "Textbooks/SPM.pdf"
REGION = "ap-south-1"
COURSE_ID = "CS6022"   # <-- set this per textbook
FILENAME = "SPM.pdf"

textract = boto3.client("textract", region_name=REGION)

# --- Start async Textract job ---
start = time.time()
response = textract.start_document_text_detection(
    DocumentLocation={"S3Object": {"Bucket": BUCKET, "Name": DOCUMENT}}
)
print("FILENAME =",FILENAME)
job_id = response["JobId"]
print(f"✅ Started Textract JobId: {job_id}")

# --- Wait for completion ---
while True:
    result = textract.get_document_text_detection(JobId=job_id)
    status = result["JobStatus"]
    if status in ["SUCCEEDED", "FAILED"]:
        break
    time.sleep(5)

elapsed = time.time() - start
if status == "FAILED":
    raise Exception("❌ Textract job failed!")

pages = result["DocumentMetadata"]["Pages"]
print(f"📄 Pages: {pages}")
print(f"⏱️ Time taken: {elapsed:.2f} sec")
print(f"💰 Estimated cost: ${(pages/1000)*1.5:.4f}")

# --- Collect page-wise text into docs with metadata ---
page_texts = {}
next_token = None

while True:
    if next_token:
        result = textract.get_document_text_detection(JobId=job_id, NextToken=next_token)
    else:
        result = textract.get_document_text_detection(JobId=job_id)

    for block in result["Blocks"]:
        if block["BlockType"] == "LINE":
            page_no = block["Page"]
            page_texts.setdefault(page_no, []).append(block["Text"])

    next_token = result.get("NextToken")
    if not next_token:
        break

# Build document objects (ready to send to Titan for embeddings)
docs = []
for page_no, lines in page_texts.items():
    chunk_text = "\n".join(lines)
    doc = {
        "chunk_text": chunk_text,
        "course_id": COURSE_ID,
        "filename": FILENAME,
        "page_no": page_no
    }
    docs.append(doc)

# --- Preview first 5 pages ---
print("\n📑 First 5 pages extracted:")
for d in docs[:5]:
    print(f"\nPage {d['page_no']} | Course: {d['course_id']} | File: {d['filename']}")
    print(d["chunk_text"][:400], "...")


FILENAME = SPM.pdf
✅ Started Textract JobId: ce41d655412d458abda1771970d243aee8b1fcb47f07c25ecdfed15620863448
📄 Pages: 396
⏱️ Time taken: 231.58 sec
💰 Estimated cost: $0.5940

📑 First 5 pages extracted:

Page 1 | Course: CS6022 | File: SPM.pdf
BoB HUGHES AND MIKE COTTERELL
Software
Project
Management Second Edition
Tistimating
www.mcgraw-hill.co.uk/hughes ...

Page 2 | Course: CS6022 | File: SPM.pdf
Software Project Management
(Second Edition) ...

Page 3 | Course: CS6022 | File: SPM.pdf
Software Project Management
(Second Edition)
Bob Hughes and Mike Cotterell,
School of Information Management, University of Brighton
The McGraw-Hill Companies
London
Burr Ridge, IL
New York
St Louis
San Francisco
Auckland
Bogotá Caracas
Lisbon
Madrid
Mexico
Milan
Montreal
New Delhi
Panama
Paris
San Juan
São Paulo
Singapore
Tokyo
Toronto ...

Page 4 | Course: CS6022 | File: SPM.pdf
Published by
McGraw-Hill Publishing Company
SHOPPENHANGERS ROAD, MAIDENHEAD, BERKSHIRE, SL6 2QL, ENGLAND
Telephone: +44(o) 

In [85]:
def make_page_chunks(docs, chunk_size=6, overlap=2):
    """
    Re-chunk page-level docs into multi-page chunks.
    Each chunk contains `chunk_size` pages, with `overlap` pages between chunks.
    """
    # Ensure pages are sorted
    docs_sorted = sorted(docs, key=lambda x: x["page_no"])

    new_chunks = []
    step = chunk_size - overlap
    total_pages = len(docs_sorted)

    for i in range(0, total_pages, step):
        # Select pages for this chunk
        chunk_docs = docs_sorted[i:i+chunk_size]
        if not chunk_docs:
            continue

        # Combine texts
        chunk_text = "\n\n".join(d["chunk_text"] for d in chunk_docs)

        # Metadata: first + last page
        page_start = chunk_docs[0]["page_no"]
        page_end = chunk_docs[-1]["page_no"]

        new_chunks.append({
            "chunk_text": chunk_text,
            "course_id": chunk_docs[0]["course_id"],   # same for all
            "filename": chunk_docs[0]["filename"],     # same file
            "page_range": f"{page_start}-{page_end}",
            "pages": [d["page_no"] for d in chunk_docs]
        })

    return new_chunks

# Example usage
chunked_docs = make_page_chunks(docs, chunk_size=6, overlap=2)
print(f"✅ Created {len(chunked_docs)} multi-page chunks")
for c in chunked_docs[:]:
    print(f"Chunk pages: {c['page_range']}, length={len(c['chunk_text'])} chars")


✅ Created 97 multi-page chunks
Chunk pages: 1-6, length=3159 chars
Chunk pages: 5-10, length=5704 chars
Chunk pages: 9-14, length=10492 chars
Chunk pages: 13-18, length=13310 chars
Chunk pages: 17-22, length=14129 chars
Chunk pages: 21-26, length=14491 chars
Chunk pages: 25-30, length=15321 chars
Chunk pages: 29-34, length=12528 chars
Chunk pages: 33-38, length=13568 chars
Chunk pages: 37-42, length=13716 chars
Chunk pages: 41-46, length=12544 chars
Chunk pages: 45-50, length=12730 chars
Chunk pages: 49-54, length=14228 chars
Chunk pages: 53-58, length=14591 chars
Chunk pages: 57-62, length=15423 chars
Chunk pages: 61-66, length=14447 chars
Chunk pages: 65-71, length=10503 chars
Chunk pages: 70-75, length=14014 chars
Chunk pages: 74-79, length=14289 chars
Chunk pages: 78-83, length=13774 chars
Chunk pages: 82-87, length=13389 chars
Chunk pages: 86-91, length=11503 chars
Chunk pages: 90-95, length=12682 chars
Chunk pages: 94-99, length=16610 chars
Chunk pages: 98-103, length=15775 chars

In [93]:
REGION = "ap-south-1"
MODEL_ID = "amazon.titan-embed-text-v2:0"
PRICE_PER_1K = 0.000024 # USD per 1k tokens

all_embeddings = []
total_tokens = 0
start = time.time()

for d in chunked_docs:
    body = json.dumps({"inputText": d["chunk_text"]})
    resp = bedrock_rt.invoke_model(modelId=MODEL_ID, body=body)
    result = json.loads(resp["body"].read())

    d["vector_field"] = result["embedding"]
    d["tokens"] = result.get("inputTextTokenCount", len(d["chunk_text"].split()))

    all_embeddings.append(d)
    total_tokens += d["tokens"]
    print(f"📄 Pages {d['page_range']}: {d['tokens']} tokens")

elapsed = time.time() - start
cost = (total_tokens / 1000) * PRICE_PER_1K

print("\n⏱️ Total time:", round(elapsed, 2), "seconds")
print("📊 Total tokens:", total_tokens)
print(f"💰 Estimated embedding cost: ${cost:.6f}")


📄 Pages 1-6: 893 tokens
📄 Pages 5-10: 1870 tokens
📄 Pages 9-14: 2338 tokens
📄 Pages 13-18: 2655 tokens
📄 Pages 17-22: 2830 tokens
📄 Pages 21-26: 2938 tokens
📄 Pages 25-30: 3119 tokens
📄 Pages 29-34: 2655 tokens
📄 Pages 33-38: 2870 tokens
📄 Pages 37-42: 2915 tokens
📄 Pages 41-46: 2745 tokens
📄 Pages 45-50: 2688 tokens
📄 Pages 49-54: 2934 tokens
📄 Pages 53-58: 3680 tokens
📄 Pages 57-62: 4093 tokens
📄 Pages 61-66: 3394 tokens
📄 Pages 65-71: 2345 tokens
📄 Pages 70-75: 2889 tokens
📄 Pages 74-79: 2947 tokens
📄 Pages 78-83: 2821 tokens
📄 Pages 82-87: 2851 tokens
📄 Pages 86-91: 2403 tokens
📄 Pages 90-95: 2910 tokens
📄 Pages 94-99: 3596 tokens
📄 Pages 98-103: 3532 tokens
📄 Pages 102-107: 2936 tokens
📄 Pages 106-111: 3069 tokens
📄 Pages 110-115: 2942 tokens
📄 Pages 114-120: 2150 tokens
📄 Pages 119-124: 2933 tokens
📄 Pages 123-128: 3292 tokens
📄 Pages 127-132: 2771 tokens
📄 Pages 131-136: 2551 tokens
📄 Pages 135-140: 3036 tokens
📄 Pages 139-144: 2917 tokens
📄 Pages 143-148: 2244 tokens
📄 Pages 14

In [90]:
import hashlib, re

def normalize_text(s: str) -> str:
    """Normalize text to avoid minor whitespace/case changes causing new IDs."""
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s.lower()

def make_chunk_id(course_id: str, chunk_text: str) -> str:
    """Deterministic ID based on course_id + chunk_text."""
    norm = normalize_text(chunk_text)
    raw = f"{course_id}|{norm}"
    return hashlib.blake2b(raw.encode("utf-8"), digest_size=16).hexdigest()


In [94]:
from opensearchpy.helpers import parallel_bulk

index_name = "test-auqa"

def doc_to_action(doc):
    # ✅ Keep original ID logic: course_id + chunk_text
    doc_id = make_chunk_id(doc["course_id"], doc["chunk_text"])

    return {
        "_op_type": "index",    # overwrite if exists
        "_index": index_name,
        "_id": doc_id,
        "_source": {
            "chunk_text": doc["chunk_text"],
            "course_id": doc["course_id"],
            "filename": doc["filename"],
            # Add both page_range (string) and pages (list)
            "page_range": doc.get("page_range"),
            "pages": doc.get("pages", [doc.get("page_no")]),
            "vector_field": doc["vector_field"]   # Titan embedding (1024 floats)
        }
    }

# build actions generator
actions = (doc_to_action(d) for d in all_embeddings)

# bulk insert
for ok, result in parallel_bulk(client, actions, thread_count=1, chunk_size=50):
    if not ok:
        print("❌ Failed:", result)

print("✅ Bulk upsert finished.")


✅ Bulk upsert finished.


In [95]:
MODEL_ID = "amazon.titan-embed-text-v2:0"

bedrock_rt = boto3.client("bedrock-runtime", region_name=REGION)
def get_titan_embedding(text: str):
    body = json.dumps({"inputText": text})
    resp = bedrock_rt.invoke_model(modelId=MODEL_ID, body=body)
    result = json.loads(resp["body"].read())
    return result["embedding"], result.get("inputTextTokenCount", None)

query_text = "Cost-benefit evaluation techniques"
query_vector, token_count = get_titan_embedding(query_text)

print(f"✅ Query embedded: {len(query_vector)}-dim vector | Tokens: {token_count}")


✅ Query embedded: 1024-dim vector | Tokens: 6


In [99]:
TOP_N = 5
BM25_WEIGHT = 0.3
VECTOR_WEIGHT = 0.7

# --- Run BM25 ---
bm25_resp = client.search(
    index=index_name,
    body={"size": TOP_N, "query": {"match": {"chunk_text": query_text}}}
)
bm25_hits = bm25_resp["hits"]["hits"]

# --- Run Vector ---
vector_resp = client.search(
    index=index_name,
    body={
        "size": TOP_N,
        "query": {
            "knn": {"vector_field": {"vector": query_vector, "k": TOP_N}}
        }
    }
)
vector_hits = vector_resp["hits"]["hits"]

# --- Normalize scores ---
def normalize_scores(hits):
    scores = [h["_score"] for h in hits]
    if not scores:
        return {}
    min_s, max_s = min(scores), max(scores)
    if min_s == max_s:
        return {h["_id"]: 1.0 for h in hits}
    return {h["_id"]: (h["_score"] - min_s) / (max_s - min_s) for h in hits}

bm25_norm = normalize_scores(bm25_hits)
vector_norm = normalize_scores(vector_hits)

# --- Combine ---
combined = {}
for h in bm25_hits + vector_hits:
    _id = h["_id"]
    src = h["_source"]
    bm25_s = bm25_norm.get(_id, 0.0)
    vec_s = vector_norm.get(_id, 0.0)
    hybrid_score = BM25_WEIGHT * bm25_s + VECTOR_WEIGHT * vec_s
    combined[_id] = {
        "hybrid_score": hybrid_score,
        "bm25_score": bm25_s,
        "vector_score": vec_s,
        "source": src
    }

results = sorted(combined.values(), key=lambda x: x["hybrid_score"], reverse=True)

# --- Print top results ---
print("\n🔎 Manual Hybrid Results:")
for r in results[:TOP_N]:   # ✅ only top N
    src = r["source"]
    # prefer page_range if exists
    page_info = src.get("page_range", src.get("page_no", "N/A"))
    print(f"Hybrid={r['hybrid_score']:.3f} | BM25={r['bm25_score']:.3f} | Vec={r['vector_score']:.3f}")
    print(f"Pages={page_info} | Course={src.get('course_id','N/A')} | File={src.get('filename','N/A')}")
    print(src["chunk_text"][:5], "...\n")



🔎 Manual Hybrid Results:
Hybrid=0.888 | BM25=1.000 | Vec=0.840
Pages=53-58 | Course=CS6022 | File=SPM.pdf
3.4 C ...

Hybrid=0.781 | BM25=0.271 | Vec=1.000
Pages=65-71 | Course=CS6022 | File=SPM.pdf
3.7 R ...

Hybrid=0.749 | BM25=0.638 | Vec=0.797
Pages=49-54 | Course=CS6022 | File=SPM.pdf
Chapt ...

Hybrid=0.097 | BM25=0.323 | Vec=0.000
Pages=155-160 | Course=CS6022 | File=SPM.pdf
7.7 E ...

Hybrid=0.007 | BM25=0.000 | Vec=0.010
Pages=61-66 | Course=CS6022 | File=SPM.pdf
3.6 C ...



In [100]:
# Collect top 5 page chunks into one string
context_text = "\n\n".join([r["source"]["chunk_text"] for r in results[:]])
ques_query = query_text   # the query you used
no = 20


In [101]:
prompt = f"""
You are an AI assistant specialized in **question generation and analysis**.

Your goals:
1. Generate insightful and generic questions based on the given context and user query.
   - Do not copy sentences from the context.
   - Make questions broad and meaningful, not text-specific.
   - Generate at least **{no} questions**.
2. For each generated question, classify it by:
   - **Difficulty level**:
       * Easy → simple recall/basic understanding (1-sentence answers).
       * Medium → requires moderate understanding, application, or 2–3 steps of reasoning.
       * Hard → requires deep understanding, critical thinking, multi-step reasoning.
   - **Bloom's taxonomy level**:
       * Choose one of: Remember, Understand, Apply, Analyze, Evaluate, Create.

### Input
Context:
{context_text}

User Query (Focus Topic):
{ques_query}

### Output
Return the results in **valid JSON** format, as a list of objects.
Each object must look like this:
{{
  "question": "...",
  "difficulty": "...",
  "blooms_level": "..."
}}

### Questions:
"""


In [103]:


REGION = "ap-south-1"
PROFILE_ARN = "arn:aws:bedrock:ap-south-1:850146468080:inference-profile/apac.anthropic.claude-3-5-sonnet-20241022-v2:0"

bedrock_rt = boto3.client("bedrock-runtime", region_name=REGION)

body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 500,
    "messages": [
        {"role": "user", "content": [{"type": "text", "text": prompt}]}
    ]
}

start = time.time()
resp = bedrock_rt.invoke_model(modelId=PROFILE_ARN, body=json.dumps(body))
elapsed = time.time() - start

output = json.loads(resp["body"].read())

reply = output["content"][0]["text"]

# Usage metadata (if present)
usage = output.get("usage", {})
input_tokens = usage.get("input_tokens", "N/A")
output_tokens = usage.get("output_tokens", "N/A")

print("✅ Generated Output:\n")
print(reply)

print("\n📊 Stats:")
print("  Input tokens :", input_tokens)
print("  Output tokens:", output_tokens)
print(f"  Time taken   : {elapsed:.2f} seconds")


✅ Generated Output:

{
  "questions": [
    {
      "question": "What are the key limitations of using simple net profit as a method for evaluating projects?",
      "difficulty": "Medium",
      "blooms_level": "Analyze"
    },
    {
      "question": "How does the payback period method help in project selection and what are its drawbacks?",
      "difficulty": "Easy", 
      "blooms_level": "Understand"
    },
    {
      "question": "Why might a project with lower IRR be more attractive than one with higher IRR?",
      "difficulty": "Medium",
      "blooms_level": "Analyze"
    },
    {
      "question": "What factors should be considered when selecting an appropriate discount rate for NPV calculations?",
      "difficulty": "Hard",
      "blooms_level": "Evaluate" 
    },
    {
      "question": "How does Net Present Value (NPV) account for the time value of money in project evaluation?",
      "difficulty": "Medium",
      "blooms_level": "Understand"
    },
    {
      "question